# web scraping RIP.ie with beautifulsoup

In [3]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.title == None:
        title = ''
    else:        
        title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [4]:
get_dn_page(390045)

390045 2019-06-05 00:00:00 Margaret  Kelly Avenue Grove, Ballymodan Place, Bandon,  Cork Cork


('Margaret  Kelly',
 datetime.datetime(2019, 6, 5, 0, 0),
 'Cork',
 'Avenue Grove, Ballymodan Place, Bandon,  Cork',
 "Kelly (Avenue Grove, Ballymodan Place, Bandon and late of Currivreeda West) on June 5th 2019. Margaret, beloved daughter of the late Timothy and Ellen. Sadly missed by her loving sisters Ann and Eileen, brothers Teddy and John, Margaret's Partner William, brothers-in-law, sisters-in-law, nieces, nephews, relatives and good friend Malcolm.;;Rosary on Friday evening at 7pm in St. Patrick's Church, Bandon. Requiem Mass on Saturday at 12 noon, funeral afterwards to the adjoining cemetery.;;May Margaret Rest in Peace")

In [28]:
#df = pd.read_csv('rip_dn_scrape.csv')
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

473362

In [34]:
z.to_csv('/home/damien/Downloads/test_day.csv')

## iterate over a range of ids and get info

In [ ]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
print (len(df))
ids = list(df.id)

results={}
for n in range(470000,475303):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)    
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

In [6]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

<ipython-input-6-849ec239fce7>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()


In [7]:
res

,id,name,date,county,address,notice
0,474838,BRENDAN CLIFFORD,2021-11-03,Kerry,"SLIEVEGAURA, Beaufort, Kerry / Causeway, Kerry","BRENDAN CLIFFORD, SLIEVEGAURA, BEAUFORT.;Brend..."
1,474839,Desmond (Des) Ferguson,2021-11-02,Meath,"Navan Road, Kells, Meath",The death has occurred peacefully following a ...
2,474840,John Joe Traynor,2021-11-03,Cavan,"Tievnaman, Termon, Virginia, Cavan","John Joe Traynor, Tievenaman, Virginia, Co. Ca..."
3,474841,Andrew John O'Shaughnessy,2021-11-02,Cork,"Carrig Na Muc Farm, Coachford, Cork","O'Shaughnessy, Andrew John (Carrig Na Muc Farm..."
4,474842,Sheila PHILLIPS (née Weston),2021-11-02,Dublin,"Malahide, Dublin","PHILLIPS (née Weston), Sheila (Malahide) - Nov..."
...,...,...,...,...,...,...
460,475298,Eddie O'Malley,NaT,,,
461,475299,Baby Grace Olivia O'Sullivan,2021-11-05,Cork,"Slip, Bantry, Cork",Cherished daughter of her heartbroken parents ...
462,475300,Davina O'Brien,2021-11-05,Tipperary,"Erinagh, Capparoe, Nenagh, Tipperary","Davina O'Brien, Erinagh, Capparoe, Nenagh, Nov..."
463,475301,Margaret Purcell-Collins (née Culhane),2021-11-06,Cork,"Ardnageetha, Charleville, Cork","Passed away, peacefully, in Windmill Nursing H..."


In [8]:
x = pd.concat([df,res]).reset_index(drop=True)
x=x[~x.id.duplicated(keep='first')]
print (len(df),len(res),len(x))
x.to_pickle('rip_dn_scrape.pkl')

474838 465 475303


In [9]:
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
x=x.drop_duplicates(['name','date','county'])
x=x.sort_values('id')
print (len(x))
#x.to_csv('rip_dn_scrape_processed.csv')
x.to_pickle('rip_dn_scrape_processed.pkl')

475303
455519
